In [5]:
using LinearAlgebra
using IterativeSolvers

# ------------------------------------------------------------
# Helper function: project vector orthogonal to v
function project(v::AbstractVector, x::AbstractMatrix)
    return x - v * v' * x
end

# ------------------------------------------------------------
# Helper function: project matrix-vector product operator
function M_operator(A, v, λ)
    I_mat = I(size(A, 1))  # Identity matrix of the same size as A
    x = (A - λ * I_mat)
    y = project(v, x) 
    z = project(v, y)
    return z
end

A = rand(100, 100)  # Example matrix A

100×100 Matrix{Float64}:
 0.184741   0.618856   0.994019   …  0.514531    0.0998173  0.21738
 0.251207   0.101032   0.811905      0.258912    0.116785   0.464897
 0.972026   0.432886   0.441422      0.374136    0.0410591  0.2177
 0.609259   0.0442203  0.723249      0.893006    0.265712   0.667828
 0.639083   0.249056   0.916541      0.613364    0.975715   0.994564
 0.197514   0.90021    0.139925   …  0.227556    0.228529   0.64142
 0.659755   0.264566   0.0643204     0.533033    0.806535   0.495272
 0.281475   0.400443   0.143006      0.100849    0.713198   0.370396
 0.796816   0.97136    0.552085      0.658009    0.59401    0.237605
 0.568139   0.178407   0.853189      0.00804848  0.674375   0.933249
 ⋮                                ⋱                         
 0.706233   0.956619   0.0445491     0.0826129   0.392513   0.00147089
 0.210392   0.280092   0.208973      0.0123298   0.803701   0.826644
 0.315898   0.0870911  0.267966      0.550291    0.72828    0.236486
 0.488345   0.93316

In [11]:
nev = 5
n = 100
λs = ones(nev)
R = ones(nev)

V = zeros(n, nev)
for i = 1:nev
    V[i,i] = 1.0
end


v = V[:,1]
λ = λs[1]
r = R[1]

# Define LinearOperator M
Mop = M_operator(A, v, λ)    
display(Mop)

println(Mop * v)


100×100 Matrix{Float64}:
 0.0         0.0         0.0         0.0        …   0.0         0.0
 0.251207   -0.898968    0.811905    0.904905       0.116785    0.464897
 0.972026    0.432886   -0.558578    0.958164       0.0410591   0.2177
 0.609259    0.0442203   0.723249   -0.0473954      0.265712    0.667828
 0.639083    0.249056    0.916541    0.998334       0.975715    0.994564
 0.197514    0.90021     0.139925    0.0521266  …   0.228529    0.64142
 0.659755    0.264566    0.0643204   0.385828       0.806535    0.495272
 0.281475    0.400443    0.143006    0.128478       0.713198    0.370396
 0.796816    0.97136     0.552085    0.500562       0.59401     0.237605
 0.568139    0.178407    0.853189    0.684297       0.674375    0.933249
 ⋮                                              ⋱              
 0.706233    0.956619    0.0445491   0.927355       0.392513    0.00147089
 0.210392    0.280092    0.208973    0.0559078      0.803701    0.826644
 0.315898    0.0870911   0.267966    0.86

[0.0, 0.25120732592126926, 0.9720258569085206, 0.6092589546943146, 0.639082633015325, 0.1975141294929842, 0.6597551316247832, 0.2814750100427611, 0.7968162835958386, 0.5681386058838946, 0.9222400746206615, 0.05832971364196449, 0.8972995935353189, 0.8365502808817686, 0.863259162955353, 0.640360415056688, 0.6975444361298244, 0.7517673819015616, 0.08073735173877084, 0.10002089464961039, 0.20243519414948485, 0.39341257690896114, 0.8414459036885861, 0.6527044824896141, 0.8956492385388205, 0.47185757495458824, 0.5880645451950244, 0.8168778246931659, 0.2833975492215859, 0.700506160765148, 0.7894655011895552, 0.375527003808269, 0.27106030487478094, 0.5838827191486159, 0.48639701662620705, 0.5530623965385271, 0.20330399534939825, 0.6447867753158767, 0.9988422224107953, 0.5010286679990856, 0.25371805900313404, 0.012246652852866746, 0.4067719669833133, 0.7021531146398166, 0.5657410134280888, 0.5341415522607683, 0.8605631786430407, 0.038369498533113444, 0.4644019558943885, 0.28171220405696895, 0.8

In [15]:
s, _ = minres(Mop, V, nev; tol=1e-6, maxiter=1000)

MethodError: MethodError: no method matching minres(::Matrix{Float64}, ::Matrix{Float64}, ::Int64; tol::Float64, maxiter::Int64)
The function `minres` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  minres(::Any, ::Any; kwargs...)
   @ IterativeSolvers C:\Users\Francesco\.julia\packages\IterativeSolvers\bjiMt\src\minres.jl:244
